In [13]:
import graphviz
import json
import os
from graphviz import Digraph
from graphviz import Source
import seaborn as sns

In [14]:
f = open("sim_taskgraph_iter1.json", "r")
j = f.read()

In [15]:
decoded = json.loads(j)

In [16]:
def task_to_str(tasks, task):
    result = ""
    if task["type"] == "computation":
        result = "O_{:d}_{:d}_{:d}_{:s}".format(task["opid"], task["part"], task["workerId"], task["op"])        
    else:
        result = "C_{:d}_{:d}_{:.0f}".format(get_task(tasks, task["fromTask"])["workerId"], get_task(tasks, task["toTask"])["workerId"], task["xferSize"]) 
    return result

In [17]:
def get_task(tasks, guid):
    for task in tasks:
        if task["guid"] == guid:
            return task
    return None

In [18]:
def generate_color_map(tasks, color_by):
    cases = {}
    for t in tasks:
        if t["type"] == "computation":
            cases[t[color_by]] = ""
    colors = sns.color_palette(n_colors=len(cases))
    cmap = {}
    for i, c in enumerate(cases.keys()):
        cmap[c] = "#{:2x}{:2x}{:2x}".format(int(colors[i][0]*255),int(colors[i][1]*255),int(colors[i][2]*255))
    return cmap


In [19]:
def task_log_to_dot_diagraph(task_log, color_by="workerId"):
    cmap = generate_color_map(task_log["tasks"], color_by)
    g = Digraph()
    tasks = task_log["tasks"]
    for t in tasks:
        g.node(task_to_str(tasks, t), task_to_str(tasks, t),
               style="filled", 
               fillcolor=cmap[t[color_by]] if t["type"] == "computation" else "{:s};0.5:{:s}".format(cmap[get_task(tasks, t["fromTask"])[color_by]],cmap[get_task(tasks, t["toTask"])[color_by]]),
               gradientangle="270", shape="ellipse" if t["type"] == "computation" else "box")
    for e in task_log["edges"]:
        g.edge(task_to_str(tasks, get_task(tasks, e[0])), 
               task_to_str(tasks, get_task(tasks, e[1])))
    return g

In [20]:
g = task_log_to_dot_diagraph(decoded)

In [21]:
g.render("config0worker.pv", format="png")

'config0worker.pv.png'

In [22]:
g.source

'digraph {\n\tO_0_0_4_conv1 [label=O_0_0_4_conv1 fillcolor="#1f77b4" gradientangle=270 shape=ellipse style=filled]\n\tO_1_0_6_pool2 [label=O_1_0_6_pool2 fillcolor="#ff7f e" gradientangle=270 shape=ellipse style=filled]\n\tO_2_0_1_conv3 [label=O_2_0_1_conv3 fillcolor="#2ca02c" gradientangle=270 shape=ellipse style=filled]\n\tO_3_0_3_pool4 [label=O_3_0_3_pool4 fillcolor="#d62728" gradientangle=270 shape=ellipse style=filled]\n\tO_4_0_6_conv5 [label=O_4_0_6_conv5 fillcolor="#ff7f e" gradientangle=270 shape=ellipse style=filled]\n\tO_5_0_5_conv6 [label=O_5_0_5_conv6 fillcolor="#9467bd" gradientangle=270 shape=ellipse style=filled]\n\tO_6_0_4_conv7 [label=O_6_0_4_conv7 fillcolor="#1f77b4" gradientangle=270 shape=ellipse style=filled]\n\tO_7_0_2_pool8 [label=O_7_0_2_pool8 fillcolor="#8c564b" gradientangle=270 shape=ellipse style=filled]\n\tO_8_0_6_flat [label=O_8_0_6_flat fillcolor="#ff7f e" gradientangle=270 shape=ellipse style=filled]\n\tO_9_0_4_linear9 [label=O_9_0_4_linear9 fillcolor="#1